In [1]:
import numpy as np
import json

In [2]:
x = np.array([0, 0, -0.068, -0.137, -0.137, -0.137, -0.162])
y = np.array([0, 0, 0, 0, 0.2, 0.39, 0.463])
z = np.array([0.12, 0.32, 0.509, 0.696, 0.7, 0.7, 0.687])
roll = np.array([0, 0, 0.349, 1.569, -1.57, -1.694, -1.694])
pitch = np.array([0, -0.349, 0, -1.222, -0.348, -0.362, -0.326])
yaw = np.array([0, 0, -1.571, -3.14, 0, 0.372, 0.372])

origins = np.array([x, y, z]).T
angles_rpy = np.array([roll, pitch, yaw]).T

In [17]:
def rotate_z(angles_rpy):
    gamma = angles_rpy[0]
    beta = angles_rpy[1]
    alpha = angles_rpy[2]
    
    sin = np.sin
    cos = np.cos
    
    z = np.array([cos(alpha)*sin(beta)*cos(gamma) + sin(alpha)*sin(gamma),
                 sin(alpha)*sin(beta)*cos(gamma) - cos(alpha)*sin(gamma),
                 cos(beta)*cos(gamma)])
    
    return z / np.linalg.norm(z)


def build_jacobian(origins, angles_rpy):
    j0 = np.concatenate([np.cross([0, 0, 1], origins[0]), [0, 0, 1]])
    j = np.array([np.concatenate([np.cross(rotate_z(angles_rpy[i]), origin - origins[i]), rotate_z(angles_rpy[i])]) for i, origin in enumerate(origins[1:])])
    
    return np.concatenate([[j0], j]).T

In [18]:
jacobian = build_jacobian(origins, angles_rpy)

np.set_printoptions(precision=3, linewidth=120)
print(jacobian)

[[ 0.000e+00  0.000e+00 -0.000e+00  1.302e-05  3.877e-03 -1.422e-04 -3.835e-03]
 [ 0.000e+00  0.000e+00  7.295e-04 -8.942e-04 -3.820e-07  0.000e+00 -1.289e-03]
 [ 0.000e+00  0.000e+00 -0.000e+00  4.806e-06  1.910e-05 -5.160e-05  1.359e-04]
 [ 0.000e+00  0.000e+00 -3.420e-01 -3.420e-01  9.551e-05 -2.716e-04 -3.202e-01]
 [ 0.000e+00  0.000e+00 -0.000e+00  6.965e-05  1.000e+00  1.000e+00  9.404e-01]
 [ 1.000e+00  1.000e+00  9.397e-01  9.397e-01  6.139e-04  7.486e-04 -1.149e-01]]


In [5]:
jacobian_inv = np.linalg.inv(jacobian[:,1:])
print(jacobian_inv)

[[-7.188e+01  1.149e+01  4.074e+03  2.773e+00  2.000e-01  1.000e+00]
 [-1.995e+01  6.191e+02  1.146e+03 -1.604e+00  5.584e-02 -0.000e+00]
 [ 8.759e+01 -6.299e+02 -4.976e+03 -1.344e+00 -2.447e-01 -0.000e+00]
 [ 1.821e+02  1.269e+01  3.790e+03  2.707e-02  2.215e-01  0.000e+00]
 [-1.143e+02 -2.352e+01 -7.644e+03 -5.017e-02  5.893e-01 -0.000e+00]
 [-7.210e+01  1.156e+01  4.099e+03  2.467e-02  2.012e-01  0.000e+00]]


In [26]:
force = np.concatenate([rotate_z(angles_rpy[-1]), [0, 0, 0]])
jacobian_inv.dot(force)

array([-440.234,  454.353,  -40.453, -488.387,  904.938, -442.966])